In [ ]:
import sys, os
sys.path.append('..') 
os.getcwd()

In [ ]:
import pandas as pd
from utils.prompts import render
from utils.llm_client import LLMClient
from utils.router import pick_model
from utils.logging_utils import log_llm_call

In [ ]:
# Load the sample messages
input_file_path = '../data/Sample Messages.txt'
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

try:
    with open(input_file_path, 'r') as f:
        messages = [line.strip() for line in f.readlines() if line.strip()]
    print(f"Loaded {len(messages)} messages.")

    print("First 5 messages:")
    for msg in messages[:5]:
        print(msg)
    
except FileNotFoundError:
    print(f"Error: File not found at {input_file_path}")
    messages = []

##### *Setup LLM Client*

In [ ]:
model = pick_model("openai", "general")
llm = LLMClient("openai", model)

##### *Examples for Rescue, Supply, Info, Other classifications*

In [ ]:
examples = """
Example 1:
        Message: "We are stuck in our house with rising flood water up to waist level in Hanwella"
        District: Colombo | Intent: Rescue | Priority: High

Example 2:
        Message: "Need baby formula and diapers for displaced families at the school shelter"
        District: Kandy | Intent: Supply | Priority: High

Example 3:
        Message: "WARNING: Dam at Moragahakanda reservoir is at risk of breaching within the next 2 hours. Immediate evacuation ordered for downstream villages in Kalu Ganga."
        District: Matale | Intent: Info | Priority: High

Example 4:
        Message: "Heart attack patient in Batticaloa needs immediate ambulance. No transport available due to flooding and landsliding."
        District: Batticaloa | Intent: Other | Priority: High

Example 5:
        Message: "Reporting an old landslide blockage in Kelanimulla from previous rains. Not blocking anything urgent"
        District: Colombo | Intent: Rescue | Priority: Low

Example 6:
        Message: "Message: For future preparedness along Attanagalu Oya, we need sandbags and tarpaulin sheets for storage."
        District: Gampaha | Intent: Supply | Priority: Low

Example 7:
        Message: "Heavy rains expected to continue for 24 hours"
        District: None | Intent: Info | Priority: Low

Example 8:
        Message: "Road from Kandy to Nuwara Eliya is now open after landslide clearance",
        District: None | Intent: Other | Priority: Low
"""

In [ ]:
##### * *

In [ ]:
results = []

print(f"{'District':<15} | {'Intent':<15} | {'Priority':<10} | {'Message':<50}")
print("-" * 80)

for msg in messages:
    prompt_text, spec = render(
                                'few_shot.v1',
                                role='Crisis Intelligence Classifier',
                                examples=examples,
                                query=f'Message: "{msg}"',
                                constraints='Classify the message exactly according to the Output format. Do not add extra text.',
                                format='District: [Name] | Intent: [Category] | Priority: [High/Low]'
                                )

    response = llm.chat([{'role': 'user', 'content': prompt_text}], temperature=0.0)
    
    output_text = response['text'].strip()
    
    parsed_data = {
                    'original_message': msg, 
                    'raw_output': output_text,
                    'district': 'Unknown',
                    'intent': 'Unknown',
                    'priority': 'Unknown'
                    }
    
    try:
        parts = [p.split(':')[1].strip() for p in output_text.split('|')]
        if len(parts) == 3:
            parsed_data['district'] = parts[0]
            parsed_data['intent'] = parts[1]
            parsed_data['priority'] = parts[2]
    except Exception as e:
        print(f"Parsing error for message '{msg}': {e}")

    results.append(parsed_data)

    log_llm_call('openai', model, 'few_shot_classify', response['latency_ms'], response['usage'])
    
    print(f"{str(parsed_data['district']):<15} | {str(parsed_data['intent']):<15} | {str(parsed_data['priority']):<10} | {msg[:30]}...")

##### *Save to an Excel*

In [ ]:
df = pd.DataFrame(results)
output_path = f'{output_dir}/classified_messages.xlsx'

df.to_excel(output_path, index=False)
print(f"\nClassified messages saved to: {output_path}")
print(df[['original_message', 'intent', 'priority']].head())

##### *Success Check*

In [ ]:
# Test example 1
test_msg1 = 'Breaking News: Kelani River level at 9m.'
prompt_text1, _ = render(
                        'few_shot.v1',
                        role='Crisis message classifier for Sri Lanka DMC',
                        examples=examples,
                        query=f'Message: {test_msg1}',
                        constraints='''
                        - Extract district name if explicitly mentioned, else None
                        - Intent: Rescue (SOS/help needed), Supply (donations/requests), Info (news/updates), Other (irrelevant/spam)
                        - Priority: High for urgent Rescue/Supply, Low otherwise
                        - If no district, use None
                        ''',
                        format='District: [Name] | Intent: [Category] | Priority: [High/Low]'
                        )

response1 = client.chat([{'role': 'user', 'content': prompt_text1}], temperature=0.0)
print('Test 1:', response1['text'])
# Expected: District: Colombo | Intent: Info | Priority: Low (note: example has Colombo, but msg doesn't; adjust if needed)

# Test example 2
test_msg2 = 'We are trapped on the roof with 3 kids!'
prompt_text2, _ = render(
    'few_shot.v1',
    role='Crisis message classifier for Sri Lanka DMC',
    examples=examples,
    query=f'Message: {test_msg2}',
    constraints='''
    - Extract district name if explicitly mentioned, else None
    - Intent: Rescue (SOS/help needed), Supply (donations/requests), Info (news/updates), Other (irrelevant/spam)
    - Priority: High for urgent Rescue/Supply, Low otherwise
    - If no district, use None
    ''',
    format='District: [Name] | Intent: [Category] | Priority: [High/Low]'
)

response2 = client.chat([{'role': 'user', 'content': prompt_text2}], temperature=0.0)
print('Test 2:', response2['text'])
# Expected: District: None | Intent: Rescue | Priority: High
